In [1]:
from gvuln import *

torch.set_default_dtype(torch.float32)

# Bounds Analysis

## Festschrift example

In [12]:
eps_list = torch.tensor([100, 0.1])

n_samples = 100_000

for eps in eps_list:

    print('\n Eps: ', eps.item())

    # computes the noise
    delta = torch.tensor(0.1)
    pars = {}
    for i in range(1, 5):
        Delta = torch.tensor(2./i)
        pars['var{}'.format(i)] = torch.sqrt(2*Delta**2*torch.log(1.25/delta)/eps**2)

    # computes the outputs in SOGA
    compiledFile = compile2SOGA('Festschrift.soga')
    cfg = produce_cfg(compiledFile)
    output_dist = start_SOGA(cfg, params_dict=pars)
    
    compiledFile_obs = compile2SOGA('Festschrift_obs.soga')
    cfg_obs = produce_cfg(compiledFile_obs)
    output_dist_obs = start_SOGA(cfg_obs, params_dict=pars)

    # computes metrics
    inc_marg = extract_marginal(output_dist, ['inc[0]'])
    inc_marg = aggregate_mixture(inc_marg)
    print('Entropy income[0]: ', entropy_gaussian(inc_marg.gm.mu[0], inc_marg.gm.sigma[0]), sample_entropy(inc_marg, n_samples),)

    inc_marg_obs = extract_marginal(output_dist_obs, ['inc[0]'])
    inc_marg_obs = aggregate_mixture(inc_marg_obs)
    print('Entropy income[0] | output == 9: ', entropy_lb(inc_marg_obs), sample_entropy(inc_marg_obs, n_samples), entropy_ub(inc_marg_obs))

    inc_out_marg = extract_marginal(output_dist, ['inc[0]', 'output'])
    inc_out_marg = aggregate_mixture(inc_out_marg)
    print('Conditional Entropy income[0] | output: lb:', cond_entropy_lb(inc_out_marg, idx_o=[1]), ' , exact: ', sample_cond_entropy(inc_out_marg, idx_o=[1], n_samples=n_samples), ' ,ub: ', cond_entropy_ub(inc_out_marg, idx_o=[1]))

    print('Mutual Information income[0] & output: ', mi_lb(inc_out_marg, idx_o=[1]), sample_mi(inc_out_marg, idx_o=[1], n_samples=n_samples), mi_ub(inc_out_marg, idx_o=[1]))
    
    print('KL Divergence income[0] || output: ', kl_div_lb(inc_marg_obs, inc_marg), sample_kl(inc_marg_obs, inc_marg, n_samples), kl_div_ub(inc_marg_obs, inc_marg))


 Eps:  100.0
Entropy income[0]:  tensor(1.4189) 1.4212324619293213
Entropy income[0] | output == 9:  1.1996612548828125 1.3780773878097534 2.443889617919922
Conditional Entropy income[0] | output: lb: -0.24372577667236328  , exact:  1.307285189628601  ,ub:  3.007410168647766
Mutual Information income[0] & output:  -1.5884716510772705 0.10452938079833984 1.6626642942428589
KL Divergence income[0] || output:  -1.0876597166061401 0.16217371821403503 0.3392512798309326

 Eps:  0.10000000149011612
Entropy income[0]:  tensor(1.4189) 1.418535590171814
Entropy income[0] | output == 9:  1.265385627746582 1.417109727859497 2.759429454803467
Conditional Entropy income[0] | output: lb: -0.1809234619140625  , exact:  1.4263386726379395  ,ub:  3.4237093925476074
Mutual Information income[0] & output:  -2.004770874977112 0.0011143684387207031 1.599861979484558
KL Divergence income[0] || output:  -1.4939805269241333 -7.3805750844258e-07 0.15342676639556885
